## **Parte 3 — `@classmethod` y `@staticmethod`**

## *1. Diferencias clave entre `@classmethod`, `@staticmethod` y métodos normales*

- **Método normal:**

&nbsp;&nbsp;&nbsp;&nbsp;Recibe como primer parámetro `self` -> trabaja sobre una **instancia**.

In [1]:
class A:
    def foo(self):  # usa self
        print("Instancia:", self)
a = A()
a.foo()  # Instancia: <__main__.A object at 0x7f...>

Instancia: <__main__.A object at 0x7d88385212b0>


- `@classmethod`:

&nbsp;&nbsp;&nbsp;&nbsp;Recibe como primer parámetro `cls` -> trabaja sobre la **clase**, no sobre una instancia.

&nbsp;&nbsp;&nbsp;&nbsp;Muy usado como **constructor alternativo** (simula “sobrecarga de constructores” que en C++ sí existe).

In [ ]:
class A:
    @classmethod
    def bar(cls):
        print("Clase:", cls)

a = A()
a.bar()  # Clase: <class '__main__.A'>  


Clase: <class '__main__.A'>


- `@staticmethod`:

&nbsp;&nbsp;&nbsp;&nbsp;No recibe ni `self` ni `cls`.

&nbsp;&nbsp;&nbsp;&nbsp;Es solo una función dentro de la clase, usada por organización.

&nbsp;&nbsp;&nbsp;&nbsp;No depende ni de la clase ni de la instancia.

In [3]:
class A:
    @staticmethod
    def baz(x, y):
        return x + y

a = A()
print(a.baz(3, 5))  # 8

8


## *2. Aplicación al proyecto: clase `Book`*

Queremos poder crear libros de diferentes maneras:
- Constructor normal (`__init__`).
- Desde un diccionario (`from_dict`).
- Un libro vacío (`empty`).
- Validaciones con un método estático auxiliar.

In [16]:
from __future__ import annotations
from typing import ClassVar

class Book:
    DEFAULT_CURRENCY: ClassVar[str] = "USD"

    def __init__(self, title: str, author: str, price: float) -> None:
        self.title = title
        self.author = author
        self.price = float(price)

    # --- Constructor alternativo desde un dict ---
    @classmethod
    def from_dict(cls, data: dict[str, str | float]) -> Book:
        """Crea un libro a partir de un diccionario con keys title, author, price"""
        return cls(data["titl"], data["author"], float(data["price"]))
    
    @classmethod
    def from_title_price(cls, title: str, price: float) -> Book:
        """Crea un libro con autor 'Unknown' por defecto"""
        return cls(title, "Unknown", price)

    # --- Constructor alternativo vacío ---
    @classmethod
    def empty(cls) -> Book:
        """Crea un libro vacío con valores por defecto"""
        return cls("Untitled", "Unknown", 0.0)

    # --- Método estático auxiliar ---
    @staticmethod
    def validate_price(price: float) -> bool:
        """Valida que el precio sea correcto (no negativo)."""
        return price >= 0


In [14]:
# Constructor normal
b1 = Book("Clean Code", "Robert C. Martin", 39.99)

# Constructor alternativo desde diccionario
data = {"titl": "Refactoring", "author": "Martin Fowler", "price": 45.50}
b2 = Book.from_dict(data)

# Constructor alternativo vacío
b3 = Book.empty()

# Validación de precio con método estático
print(Book.validate_price(10.0))   # True
print(Book.validate_price(-5.0))   # False


True
False


In [17]:
b4 = Book.from_title_price("The Pragmatic Programmer", 42.0)
assert b4.author == "Unknown"

## *3. Explicación*

- `from_dict` usa `cls(...)`. Esto significa que si más adelante heredas de Book,
ese constructor alternativo creará instancias de la subclase automáticamente. (Esto es lo que lo diferencia de un simple `staticmethod`).

- `empty` es una forma explícita de crear un objeto con valores predefinidos. En C++ esto sería un **constructor sobrecargado sin parámetros**.

- `validate_price` es un método **independiente del estado**.
Podría estar fuera de la clase, pero mantenerlo dentro la hace más organizada.

- En c++

```cpp
class Book {
public:
    string title, author;
    double price;

    // Constructor normal
    Book(string t, string a, double p): title(t), author(a), price(p) {}

    // Constructor vacío
    Book(): title("Untitled"), author("Unknown"), price(0.0) {}

    // Método estático
    static bool validate_price(double p) {
        return p >= 0;
    }
};

```

## *4. `@classmethod` vs `@staticmethod` en herencia*

> Cuando defines un método de clase:

In [ ]:
class Book:
    def __init__(self, title, author, price):
        self.title = title
        self.author = author
        self.price = price

    @classmethod
    def from_dict(cls, data: dict):
        return cls(data["title"], data["author"], data["price"])


- El primer parámetro es `cls`, que representa **la clase sobre la que se llama el método.**
- Eso significa que si heredas de `Book`, el `cls` ya no será `Book`, sino la **subclase**.
- Y como devolvemos `cls(...)`, lo que se crea es una instancia de esa **subclase**, no de la clase base.

> Ejemplo práctico:

In [24]:
class Book:
    def __init__(self, title, author, price):
        self.title = title
        self.author = author
        self.price = price

    @classmethod
    def from_dict(cls, data: dict):
        return cls(**data)   # desempaqueta todo el dict como argumentos


# Subclase
class Ebook(Book):
    def __init__(self, title, author, price, filesize):
        super().__init__(title, author, price)
        self.filesize = filesize


# Uso
data = {"title": "Refactoring", "author": "Martin Fowler", "price": 45.50}
b1 = Book.from_dict(data)   # aquí cls = Book
print(type(b1))             # <class '__main__.Book'>

data2 = {"title": "Clean Code", "author": "Robert C. Martin", "price": 39.99, "filesize": 2.5}
b2 = Ebook.from_dict(data2)  # aquí cls = Ebook
print(type(b2))             # <class '__main__.Ebook'>




<class '__main__.Book'>
<class '__main__.Ebook'>
